# Ch04 - NaiveBayes 예습

In [2]:
import os
srcDir = "C:\Users\suh\Desktop\machinelearninginaction"
ch4dd = os.path.join(srcDir, 'Ch04')
%cd {ch4dd}

C:\Users\suh\Desktop\machinelearninginaction\Ch04


In [4]:
import bayes

def loadDataSet():
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1] # 1:폭력적인, 0:폭력적이지 않음
    return postingList, classVec
    
# 유일한 단어 집합으로 변환하는 함수
def createVocabList(dataSet):
    vocabSet = set([])
    for document in dataSet:
        vocabSet = vocabSet | set(document)
    return list(vocabSet)


# 문서(vocalList) 내에 어휘 목록(inputSet)의 단어가 존재하는지 여부를 0과 1의 배열로 생성하는 함수
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else:
            print "the word: %s is not in my Vocabulary!" % word
        return returnVec

    
listOPosts, listClasses = loadDataSet()
myVocabList = createVocabList(listOPosts)
print myVocabList

['cute', 'love', 'help', 'garbage', 'quit', 'I', 'problems', 'is', 'park', 'stop', 'flea', 'dalmation', 'licks', 'food', 'not', 'him', 'buying', 'posting', 'has', 'worthless', 'ate', 'to', 'maybe', 'please', 'dog', 'how', 'stupid', 'so', 'take', 'mr', 'steak', 'my']


In [97]:
print bayes.setOfWords2Vec(myVocabList, listOPosts[0])
print bayes.setOfWords2Vec(myVocabList, listOPosts[1])
print bayes.setOfWords2Vec(myVocabList, listOPosts[2])
print bayes.setOfWords2Vec(myVocabList, listOPosts[3])
print bayes.setOfWords2Vec(myVocabList, listOPosts[4])
print bayes.setOfWords2Vec(myVocabList, listOPosts[5])

[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0]
[1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]


In [6]:
from numpy import *


#input : 문서 행렬(trainMatrix) => [[문서1], [문서2], ...]
#        문서에 대한 분류 항목(trainCategory) => [0,1,0,1,0,1]
def trainNB0(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    
    pAbusive = sum(trainCategory) / float(numTrainDocs) # (폭력적 문서 갯수) / (전체 문서 갯수)
    
    p0Num = zeros(numWords); p1Num = zeros(numWords) # 분자 초기화
    p0Denom = 0.0; p1Denom = 0.0 # 분모 초기화
    
    # 문서 전체 탐색
    for i in range(numTrainDocs):
        if trainCategory[i] == 1: # 그 문서가 폭력적 문서이면.
            p1Num += trainMatrix[i] # p1Num배열에서 해당 문서의 단어를 포함하는 인덱스 +1
            p1Denom += sum(trainMatrix[i]) # 해당 문서 내 단어 총갯수
        else: # 폭력적 문서 아니면.
            p0Num += trainMatrix[i] # p1Num배열에서 해당 문서의 단어를 포함하는 인덱스 +1
            p0Denom += sum(trainMatrix[i]) # 해당 문서 내 단어 총갯수
    
    print "p0Num : %s" % p0Num
    print "p1Num : %s" % p1Num
    p1Vect = p1Num / p1Denom # 폭력적 단어 확률
    p0Vect = p0Num / p0Denom # 비폭력적 단어 확률
    
    return p0Vect, p1Vect, pAbusive
   
reload(bayes)
listOPosts, listClasses = bayes.loadDataSet()

myVocabList = bayes.createVocabList(listOPosts)

trainMat = []
for postinDoc in listOPosts:
    temp = bayes.setOfWords2Vec(myVocabList, postinDoc)
    print temp
    trainMat.append(temp)
p0V, p1V, pAb = trainNB0(trainMat, listClasses)

#print "p0V : %s" % p0V
#print "p1V : %s" % p1V
#print "pAb : %s" % pAb


#print trainMat
#print listClasses

[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0]
[1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
p0Num : [ 1.  1.  1.  0.  0.  1.  1.  1.  0.  1.  1.  1.  1.  0.  0.  2.  0.  0.
  1.  0.  1.  1.  0.  1.  1.  1.  0.  1.  0.  1.  1.  3.]
p1Num : [ 0.  0.  0.  1.  1.  0.  0.  0.  1.  1.  0.  0.  0.  1.  1.  1.  1.  1.
  0.  2.  0.  1.  1.  0.  2.  0.  3.  0.  1.  0.  0.  0.]


In [111]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + log(pClass1)
    p0 = sum(vec2Classify * p0Vec) + log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else:
        return 0
    
def testingNB():
    listOPosts, listClasses = bayes.loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat=[]
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    p0V, p1V, pAb = bayes.trainNB0(array(trainMat), array(listClasses))

    testEntry = ['love', 'my', 'dalmation']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print testEntry, 'classified as: ', classifyNB(thisDoc, p0V, p1V, pAb)

    testEntry = ['stupid', 'garbage']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print testEntry, 'Classified as: ', classifyNB(thisDoc, p0V, p1V, pAb)

        
bayes.testingNB()

['love', 'my', 'dalmation'] classified as:  0
['stupid', 'garbage'] classified as:  1


* 예제#1 - 스팸 이메일 분류하기

In [115]:
mySent = 'This book is the best book on Python or M.L. I have ever laid eyes upon.'
mySent.split()

['This',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'Python',
 'or',
 'M.L.',
 'I',
 'have',
 'ever',
 'laid',
 'eyes',
 'upon.']

In [129]:
import re
regEx = re.compile('\\W*')
emailText = open('email/ham/6.txt').read()
print emailText
listOfTokens = regEx.split(emailText)
print listOfTokens

Hello,

Since you are an owner of at least one Google Groups group that uses the customized welcome message, pages or files, we are writing to inform you that we will no longer be supporting these features starting February 2011. We made this decision so that we can focus on improving the core functionalities of Google Groups -- mailing lists and forum discussions.  Instead of these features, we encourage you to use products that are designed specifically for file storage and page creation, such as Google Docs and Google Sites.

For example, you can easily create your pages on Google Sites and share the site (http://www.google.com/support/sites/bin/answer.py?hl=en&answer=174623) with the members of your group. You can also store your files on the site by attaching files to pages (http://www.google.com/support/sites/bin/answer.py?hl=en&answer=90563) on the site. If you�re just looking for a place to upload your files so that your group members can download them, we suggest you try Googl

In [141]:
bayes.spamTest()

classification error ['home', 'based', 'business', 'opportunity', 'knocking', 'your', 'door', 'don', 'rude', 'and', 'let', 'this', 'chance', 'you', 'can', 'earn', 'great', 'income', 'and', 'find', 'your', 'financial', 'life', 'transformed', 'learn', 'more', 'here', 'your', 'success', 'work', 'from', 'home', 'finder', 'experts']
classification error ['yeah', 'ready', 'may', 'not', 'here', 'because', 'jar', 'jar', 'has', 'plane', 'tickets', 'germany', 'for']
the error rate is:  0.2
